In [55]:
'''
BY @shan1322
'''
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import os
import io
import sys

In [56]:
#loading and preprocessing data
path="shakespeare"
files=os.listdir(path)
script=""
for i in files:
    with io.open(path+"/"+i, encoding='utf-8') as f:
        text = f.read().lower()
    script=script+text
chars = sorted(list(set(script)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))
maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(script) - maxlen, step):
    sentences.append(script[i: i + maxlen])
    next_chars.append(script[i + maxlen])
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [ ]:

# build the model: a single LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


def on_epoch_end(epoch, logs):
    # Function invoked at end of each epoch. Prints generated text.
    print()
    print(' Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print('diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print(' Generating with seed: "' + sentence + '"')
        sys.stdout.write(generated)

        for i in range(400):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=5,
          callbacks=[print_callback])

Epoch 1/5
1761252/1761252 [==============================] - 1888s 1ms/step - loss: 1.7087

 Generating text after Epoch: 0
diversity: 0.2
 Generating with seed: "
	and arms her with the boldness of a wi"

	and arms her with the boldness of a will the seek
	the mort and the soul the come the man the lord
	the season of my this the lord to the hand
	the king of the scame the comparied of the stard,
	the seek and here the sword, i do have thee,
	the bose of the fair from the serving,
	the man and the seek the world the come,
	and the man and the seek the serve the seek
	the seek and the stand to the seek the seek
	and the deed the seek the
diversity: 0.5
 Generating with seed: "
	and arms her with the boldness of a wi"

	and arms her with the boldness of a will
	have be the scaminade, which a desaries.
	the bare and boy his peroud lead the ephe
	drink and heaven i have speek and his
	i forth, i will the came?

	[enter sir o, at so much arm, and hell thee and wors
	and such a out in the a